# Raw data preprocessing notebook

Create a variable with data directory

In [ ]:
val dataDir = System.getenv("HOME") + "/data/history"
val spark   = sparkSession

### Read csv file to Dataframe

* No header
* Infer a schema
* name columns


In [ ]:
val rawDF = spark.read.format("csv")
        .option("header", "false")
        .option("inferSchema", "true")
        .load(s"${dataDir}/*.csv")
        .toDF("instrument","timestamp","open","high","low","close","volume")

In [ ]:
rawDF.count

### Find distinct instruments

In [ ]:
rawDF.select("instrument").distinct

### Count number of distinct timestamps

In [ ]:
// ...

### Count number of timestamp for each instrument

In [ ]:
// ...

### Count number of occurences of each timestamps and display distribution

In [ ]:
rawDF.groupBy("timestamp").count
     .toDF("ts", "ts_count")
     .groupBy("ts_count").count
     .orderBy($"ts_count".asc)

### Data cleaning: only keep timestamps with the 5 instruments

These counts teach us there are occurences of `timestamps` with duplicate or missing `instruments`

We need to remove any duplicate line then identify timestamps with the 5 instruments

In [ ]:
(rawDF.count, rawDF.distinct.count)

In [ ]:
rawDF.distinct
     .groupBy("timestamp").count
     .toDF("ts", "ts_count")
     .groupBy("ts_count").count
     .orderBy($"ts_count".asc)

In [ ]:
rawDF.distinct
     .groupBy("timestamp").agg(collect_list($"instrument"))
     .toDF("ts", "instruments")
     .count

In [ ]:
rawDF.distinct
     .groupBy("timestamp").agg(collect_list($"instrument"))
     .toDF("ts", "instruments")
     .filter("size(instruments) != 5")
     .count

In [ ]:
val timestamps = rawDF.distinct
     .groupBy("timestamp").agg(collect_list($"instrument"))
     .toDF("ts", "instruments")
     .filter("size(instruments) == 5")
     .select("ts")

There are 10067 `timestamps` to keep to get only complete data

In [ ]:
timestamps.count

### Inner join `timestamps` to filter data

In [ ]:
val filtered = rawDF.distinct
                    .join(timestamps, $"timestamp" === $"ts")
                    .select("timestamp", "instrument", "close")
//.count

In [ ]:
50385/5

In [ ]:
filtered.distinct.count
     //.groupBy("timestamp").agg(collect_list($"instrument"))
     //.toDF("ts", "instruments")
     //.count

In [ ]:
50335/5

### Pivot the table to get instruments as colums

* Aggregate by timestamp to define the rows keys
* Pivot around instruments to define columns
* Keep the min (or max or avg -- only one element is used anyway thanks to previous filtering)
* Order by timestamp

In [ ]:
val data = filtered.groupBy($"timestamp")   
                   .pivot($"instrument")
                   .agg(min("close"))
                   .orderBy($"timestamp".desc)

### Save as parquet file

See partitions on disk

In [ ]:
// TODO ... toon many partitions ...

In [ ]:
val dataLocation = System.getenv("HOME") + "/data/cleaned-history.parquet"
data.write.save(dataLocation)

In [ ]:
:sh ls -l $dataLocation